In [ ]:
import numpy as np

import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [ ]:
def load_mfcc_files(directory):
    mfcc_data = []
    for file in os.listdir(directory):
            print(file)
            file_path = os.path.join(directory, file)
            data = pd.read_csv(file_path, header=None).values
            mfcc_data.append(data)
            print(data.shape)
    return mfcc_data

# Load MFCC data
asha = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Asha Bhosle")
bhavgeet = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Bhavgeet")
kishor = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Kishor Kumar")
lavni = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Lavni")
michael = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Michael Jackson")
national = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\National Anthem")

# Debug: Print the number of files loaded from each directory
print(f"Asha Bhosle: {len(asha)} files")
print(f"Bhavgeet: {len(bhavgeet)} files")
print(f"Kishor Kumar: {len(kishor)} files")
print(f"Lavni: {len(lavni)} files")
print(f"Michael Jackson: {len(michael)} files")
print(f"National Anthem: {len(national)} files")

In [ ]:
#repeat songs

asha = np.array(asha)
bhavgeet = np.array(bhavgeet)
kishor=np.array(kishor)
lavni=np.array(lavni)
michael=np.array(michael)
national=np.array(national)
print(asha.shape)
print(bhavgeet.shape)
print(kishor.shape)
print(lavni.shape)
print(michael.shape)
print(national.shape)

asha=asha[:-1]
bhavgeet = bhavgeet[:-1]
kishor=kishor[:-1]
lavni=lavni[:-3]

michael=michael[:-1]
print("new")
print(asha.shape)
print(bhavgeet.shape)
print(kishor.shape)
print(lavni.shape)
print(michael.shape)
print(national.shape)


In [ ]:

# Combine data
data = np.concatenate((asha, bhavgeet, kishor, lavni, michael, national), axis=0)

# Create labels
labels = ['asha'] * len(asha) + ['bhavgeet'] * len(bhavgeet) + ['kishor'] * len(kishor) + ['lavni'] * len(lavni) + ['michael'] * len(michael) + ['national'] * len(national)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Combine data and labels into a single array
combined_data = np.empty((data.shape[0], data.shape[1], data.shape[2] + 1), dtype=object)
combined_data[:, :, :-1] = data
combined_data[:, 0, -1] = encoded_labels  # Add labels in the last column

# Check the shape of the combined data
print(f"Combined data shape: {combined_data.shape}")

#The shape of combined_data will be (294, 20, 25001), where the last element in the innermost dimension contains the encoded labels.

In [ ]:
# Extract the data without the labels
X = combined_data[:, :, :-1]

# Extract the labels
y = combined_data[:, 0, -1]

# Print the shapes of X and y to verify
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
X = X.astype(np.float32)
y = y.astype(np.int32)
print(y)


In [ ]:

label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y)

# Print the encoded labels to verify
print(f"Encoded labels: {encoded_y}")

In [ ]:

# Step 1: Split the dataset into training and test sets (80% training, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.2, random_state=42)

# Step 2: Split the training set into training and validation sets (80% training, 20% validation)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
print(x_val[:1])
print(x_val[:1].shape)


In [ ]:
import numpy as np
from tensorflow.keras import layers, models, callbacks
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming x_train, x_val, x_test, y_train, y_val, y_test are already defined
# and have the shape (176, 20, 25000)

# Build the CNN model for feature extraction
def build_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv1D(32, 5, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2, padding='same'))
    model.add(layers.Dropout(0.25))
    
    

    #model.add(layers.Conv1D(64, 3, activation='relu', padding='same'))
    #model.add(layers.BatchNormalization())
    #model.add(layers.MaxPooling1D(2, padding='same'))
    #model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv1D(128, 5, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2, padding='same'))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, activation='softmax'))  # Assuming 6 classes for classification
    return model

input_shape = (20, 25000)  # Adjusted to match the input data shape
cnn_model = build_cnn_model(input_shape)

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the data to match the input shape of the model
x_train = x_train.reshape(-1, 20, 25000)
x_val = x_val.reshape(-1, 20, 25000)
x_test = x_test.reshape(-1, 20, 25000)

# Define early stopping callback
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
# Train the CNN model with early stopping
cnn_model.fit(x_train, y_train, epochs=200, batch_size=16, validation_data=(x_val, y_val), callbacks=[early_stopping,lr_scheduler])


# Assuming X_test and y_test are your test data and labels
X_test = x_test.astype(np.float32)
y_test = y_test.astype(np.int32)  # Ensure y_test is integer type for sparse_categorical_crossentropy

# Evaluate the model on the test data
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test, batch_size=32)

# Print the test accuracy
print(f"Test accuracy: {test_accuracy}")
cnn_model.summary()

# Predict the labels for the test set
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels if necessary

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print detailed classification report
print(classification_report(y_test, y_pred_classes))
